In [6]:
import torch
from model import CNNLSTM  # Replace with your actual model class name if different
from dataset import DebrisFlowDataset  # Replace with your actual dataset class name if different
from train import Trainer
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)


cpu


In [7]:

# Load the complete dataset
full_dataset = DebrisFlowDataset(r'C:\Users\thomas.bush\repos\dyna-landslide-surrogate\data_temp', 5)

# Define the size of the train and validation datasets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Perform the split
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


In [8]:
# Define model
model = CNNLSTM(in_channels=3, lstm_hidden_size=1500)  # Replace CNNLSTM with your actual model class name

# Check if multiple GPUs are available and wrap the model using nn.DataParallel
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    # This will wrap the model for use with multiple GPUs
    model = torch.nn.DataParallel(model)

model = model.to(device)


# Define loss function and optimizer
criterion = torch.nn.MSELoss()  # Choose the appropriate loss function for your problem
optimizer = Adam(model.parameters(), lr=1e-3)

# Initialize Trainer
# Parameters for data loading
batch_size = 3
shuffle = True
num_workers = 4

# Create the DataLoader for the training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

# Create the DataLoader for the validation set
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Initialize Trainer with DataLoaders
trainer = Trainer(model=model, train_dataloader=train_loader, val_dataloader=val_loader,
                  criterion=criterion, optimizer=optimizer, device=device)









In [9]:
# Train the model
num_epochs = 10  # Set the number of epochs you wish to train for
trainer.train(num_epochs)